##  Реализация web-crawler для сбора коллекции документов с указанного ресурса. 

Выбор формата хранения коллекции - MongoDB? Forlder with files? Relation DataBase

### Характеристики коллекции:
- Объем собранной коллекции не менее 100 тысяч уникальных документов 
- Размер текста для каждого документа должен быть не менее 2000 символов
- Механизм очистки документов от не релевантной информации, например:
    - html теги
    - ссылки на сторонние или внутренние ресурсы сайта
    - ссылки на изображения и видео
    - знаки препинания
- Выделение признаков документа
    - автор
    - тематические теги
    - рейтинг
    - репосты
    - дата публикации 
    - другая метаинформация, предоставляемая ресурсом.

### Результатом сдачи является:
- код web-crawler
- инструмент очистки текстов
- исходная коллекция документов
- коллекция документов после очистки



## Результаты сейчас:

- Код для скачивания данных с сайта по дате
- Выгрузка осуществлена за последние 280 дней в формат html


In [ ]:
Признаки документа
 
После исследования содержимого для статьи (тег <article>)
В качестве важных признаков документа отобраны:
    
+ идентификатор статьи: <article data-story-id="7358483" >
    
+ ссылка на статью и заголовок статьи: <h2 class="story__title">
    <a class="story__title-link" 
    href="https://pikabu.ru/story/13_minut_kotoryie_ne_izmenili_mir_7356303" 
    target="_blank">13 минут, которые не изменили мир</a></h2>
</header>
    
+ идентификатор автора: ( <div class="user__info-item"><a class="user__nick story__user-link"
                         data-profile="true" href="/@AuthorName">AuthorName</a></div> )
    
+ дата публикации: <div class="user__info-item">
    <time class="caption story__datetime hint" datetime="2020-04-09T07:34:46+03:00">2 дня назад</time>
    </div>
    
+ рейтинг: <div class="story__rating-count">14812</div>
    
+- количчество просмотров: <div aria-label="619995 просмотров" class="story__views hint">
    <span class="story__views-count story__views-count_loaded">620K</span>
</div>

+ количество комментариев: <span class="story__comments-link-count">1046</span>
    
+ теги  <div class="story__tags tags">
    <a class="tags__tag" data-tag="Текст" data-tag-menu="true" 
        href="/tag/%D0%A2%D0%B5%D0%BA%D1%81%D1%82/hot" rel="noopener"
        target="_blank">Текст</a> 
    <a >Соседи</a> 
    <a >Электричество</a> 
    <a >Воровство</a>
</div>

+ текст публикации
    <div class="story__content-inner">
    И очень-очень много вложенных тегов разного вида
</div>

In [23]:
from bs4 import BeautifulSoup
import codecs
import re

In [15]:
def open_html(file_name):
    f = codecs.open(file_name, 'r', 'utf-8')
    document = BeautifulSoup(f.read())
    return document

def printh(text):
    print(text.prettify())

In [ ]:
printh(open_html("source_html/pikabu10.html"))

In [49]:
def tags(page):
    
    t = page.find('div', {'class': 'story__tags tags'}).text
    t = re.sub("^\s+|\n|\r|\s+$", '', t)
    t = t.split(" ")
    return t

art10 = open_html("source_html/pikabu10.html")
print(tags(art10))

['Работа', 'Юмор', '2020', 'Текст', 'Карантин']


In [53]:
def text(page):
    
    t = page.find('div', {'class': 'story__content-inner'}).text
    # clean_text(t)
    # t = t.split(" ")
    return t

print(text(art10))


Помню, как иногда, отпрашивался на работе, и сваливал незаметно, но чтобы ходить на работу тайком от президента, к этому меня жизнь не готовила...



In [58]:
def amount_comments(page):
    
    t = page.find('span', {'class': 'story__comments-link-count'}).text
    return t

print(amount_comments(art10))

495


In [59]:
def rating(page):
    
    t = page.find('div', {'class': 'story__rating-count'}).text
    return t

print(rating(art10))

10121


In [60]:
def author(page):
    
    t = page.find('div', {'class': 'user__info-item'}).text
    return t

print(author(art10))

alltaets


In [61]:
def title(page):
    
    t = page.find('h2', {'class': 'story__title'}).text
    return t

print(title(art10))

Рабочие моменты


In [70]:
def link(page):
    
    t = [item['href'] for item in page.select('h2 a')]
    return t[0]

print(link(art10))

https://pikabu.ru/story/rabochie_momentyi_7358483


In [73]:
def article_id(page):
    
    t = [item['data-story-id'] for item in page.select('article')]   
    return t[0]

print(article_id(art10))

7358483


In [118]:
def views(page):
    
    t = page.find('div', {'class': 'story__views hint'}).text
    t = re.sub("^\s+|\n|\r|\t|\s+$", '', t)
    t = t.split(" ")
    if (str(t[0])[-1] == 'K'):
        t = int(str(t[0])[:-1:]) * 1000
    return t
    
print(views(art10))

324000


In [137]:
import datetime

In [141]:
def page_date(page):
    
    s = [item['datetime'] for item in page.select('time')]  
    s = str(s[0])[:10]
    s = s.split("-")
    d = datetime.date(int(s[0]), int(s[1]), int(s[2]))
    return d

print(page_date(art10))

2020-04-09
